# Download and import librairies

In [1]:
# Import Librairies
# !pip install gspread_dataframe gspread oauth2client
# !pip install gspread-pandas
# !{sys.executable} -m pip install pygsheets
from gspread_pandas import Spread, Client
from google.oauth2.service_account import Credentials
import pandas as pd
import re
import sys
import json
import gspread
import pygsheets
import csv
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import DBSCAN

# Import our Google Search Console data from a CSV file

In [3]:
# Import our Google Search Console CSV Data into a Dataframe
df = pd.read_csv('Queries.csv', delimiter=',')
df.head(20)


,Top queries,Clicks,Impressions,CTR,Position
0,adelphi living,289,834,34.65%,1.07
1,adelphi,15,298,5.03%,9.97
2,sil providers perth,12,90,13.33%,3.26
3,supported independent living perth,7,323,2.17%,10.77
4,adelphi living mandurah,7,30,23.33%,1.03
5,disability accommodation perth,3,641,0.47%,31.52
6,ndis providers near me,3,45,6.67%,1.00
7,sda providers perth,3,23,13.04%,7.48
8,disability housing perth,2,541,0.37%,31.37
9,ndis providers perth,2,258,0.78%,52.90


## Create a list of keywords from our GSC data

In [4]:
# Define a variable to hold our 'Top queries' column only
top_queries = df['Top queries']

# Create a list of keywords and populate it from our dataframe
keywords = []

for x in top_queries:
  keywords.append(x)

# Display keywords list
keywords

['adelphi living',
 'adelphi',
 'sil providers perth',
 'supported independent living perth',
 'adelphi living mandurah',
 'disability accommodation perth',
 'ndis providers near me',
 'sda providers perth',
 'disability housing perth',
 'ndis providers perth',
 'sil ndis',
 'ndis housing perth',
 'sil',
 'disability supported accommodation perth',
 'ndis sda',
 'disability accommodation wa',
 'ndis events perth',
 'disability service providers perth',
 'ndis short term accommodation',
 'disability housing',
 'disability homes',
 'adelphi living jobs',
 'sil housing',
 'the adelphi',
 'sda payment',
 'sda living',
 'support coordinators',
 'disability services near me',
 'adelphi house assisted living',
 'ndis wa providers',
 'ndis sda application form',
 'adelphi housing',
 'disability support organisation',
 'disability support services near me',
 'independent living supplies',
 'ndis support provider',
 'ndis',
 'ndis perth',
 'sda ndis',
 'supported independent living ndis',
 'spec

# Start clustering our keywords using Machine Learning sklearn library

In [5]:
# Define our keyword clustering model with sklearn and TfidfVectorizer (using DBSCAN and not Kmean)
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=10000,min_df=0.01,use_idf=True, ngram_range=(1,2))

# Import our keyword list for Keyword Clustering
tfidf_matrix = tfidf_vectorizer.fit_transform(keywords)

# Define settings for cluster adjustment (number and size of clusters)
ds = DBSCAN(eps=0.7, min_samples=5).fit(tfidf_matrix)
clusters = ds.labels_

## Create a dataframe to hold our keyword clusters

In [6]:
# Create a dataframe for our clusters
cluster_df = pd.DataFrame(clusters, columns=['Cluster Number'])

# Merge our cluster df with our top queries df
result = pd.merge(cluster_df, top_queries, left_index=True, right_index=True)

# We group our keywords by cluster number and separate them with a comma
clusters_queries = result.groupby(['Cluster Number'])['Top queries'].apply(', '.join).reset_index()

# Extend Column and Row width for a better readability 
pd.set_option('max_colwidth', 400)
pd.set_option('max_rows', 100)

clusters_queries


,Cluster Number,Top queries
0,-1,"sil providers perth, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, sda house, sda house available, independent living homes, independent living services, sda perth, specialist support coordination perth, independent living solutions, housing disability perth, independent living disabled,..."
1,0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, supported independent living wa, sil supported independent living, supported independent living accommodation, supported independent living ndis, supported independent living vaca..."
3,2,"ndis providers near me, disability services near me, disability support services near me, independent living near me, independent living facilities near me, ndis service providers near me, disabled living near me, disability facility near me, ndis support workers near me, disability homes near me, support workers near me, assisted accommodation near me"
4,3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,5,"sil ndis, sil, sil accommodation, sil accomodation, sil sda, sil assessment ndis, sil eligibility, sil perth, sil supports, sil logo, sil living, ndia sil, sil ltd, sda sil, sil ndia, sil disability, sil insurance, sil., sil sda ndis, disability sil, www.sil, supports sil"
7,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, retirement living perth, perth living, community living perth, ndis renovations perth, living perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, simple approach perth"


## Use the same ML model on each cluster so we can name our clusters automatically

In [8]:
# Create an empty list to store our cluster names
cluster_name = []

# Use Iterrows and tfIdVectorizer to get all words and their TF-IDF Score per cluster
for index, row in clusters_queries.iterrows():
    
    queries = row['Top queries']
    queries = [queries]

    # Add a stop_words function to drop irrelevant words (ex: “a”, “the”, “is”, “are”, "and" etc...)
    my_stop_words = text.ENGLISH_STOP_WORDS

    # Run tfIdfVectorizer for each cluster
    tfIdfVectorizer=TfidfVectorizer(use_idf=True, stop_words=my_stop_words)
    tfIdf = tfIdfVectorizer.fit_transform(queries)
    query_score_cluster = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF Score"])
    query_score_cluster = query_score_cluster.sort_values('TF-IDF Score', ascending=False)

    # Get the top 3 queries (selected by highest scores) per cluster and get them into lists, clean words of commas and ''
    top_three_queries = query_score_cluster.index[:3]
    top_three_queries_list = top_three_queries.values.tolist()
    top_three_queries_list = str(top_three_queries_list).replace(',', '')
    top_three_queries_list = str(top_three_queries_list).replace('[','').replace(']','')
    top_three_queries_list = str(top_three_queries_list).replace("'","")

    # Append our top 3 queries to our cluster_name list
    cluster_name.append(top_three_queries_list)
    
    # Create a Cluster Name dataframe    
    cluster_name_df = pd.DataFrame({'Cluster Name': cluster_name})

cluster_name_df


,Cluster Name
0,living perth ndis
1,adelphi freedom perth
2,independent living supported
3,near disability living
4,providers perth sda
5,housing disability perth
6,sil ndis sda
7,accommodation supported disability
8,ndis sda application
9,perth ndis living


# Merge our dataframes so we can see all of our keywords grouped into different clusters and their names automatically generated. We can make sure it is matching properly

In [9]:
# Merge cluster_name_df with clusters_queries so we can have them matching and double check the cluster names match with our queries
clusters = pd.merge(cluster_name_df, clusters_queries, left_index=True, right_index=True)
clusters

,Cluster Name,Cluster Number,Top queries
0,living perth ndis,-1,"sil providers perth, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, sda house, sda house available, independent living homes, independent living services, sda perth, specialist support coordination perth, independent living solutions, housing disability perth, independent living disabled,..."
1,adelphi freedom perth,0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,independent living supported,1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, supported independent living disability, supported independent living wa, sil supported independent living, supported independent living accommodation, supported independent living ndis, supported independent living vaca..."
3,near disability living,2,"ndis providers near me, disability services near me, disability support services near me, independent living near me, independent living facilities near me, ndis service providers near me, disabled living near me, disability facility near me, ndis support workers near me, disability homes near me, support workers near me, assisted accommodation near me"
4,providers perth sda,3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,housing disability perth,4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,sil ndis sda,5,"sil ndis, sil, sil accommodation, sil accomodation, sil sda, sil assessment ndis, sil eligibility, sil perth, sil supports, sil logo, sil living, ndia sil, sil ltd, sda sil, sil ndia, sil disability, sil insurance, sil., sil sda ndis, disability sil, www.sil, supports sil"
7,accommodation supported disability,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,ndis sda application,7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,perth ndis living,8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, retirement living perth, perth living, community living perth, ndis renovations perth, living perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, simple approach perth"


## We only retain our Cluster names and numbers together

In [10]:
# splited_clusters = pd.concat([pd.Series(row['Cluster Name'], row['Top queries'].split(','))              
#                     for _, row in clusters.iterrows()]).reset_index()
cluster_name_nb = clusters[["Cluster Name", "Cluster Number"]]
cluster_name_nb.head()

,Cluster Name,Cluster Number
0,living perth ndis,-1
1,adelphi freedom perth,0
2,independent living supported,1
3,near disability living,2
4,providers perth sda,3


In [11]:
# Merge it with our machine learning results
clusters_queries = pd.merge(result, cluster_name_nb, how="inner", on="Cluster Number")
clusters_queries.head() 



,Cluster Number,Top queries,Cluster Name
0,0,adelphi living,adelphi freedom perth
1,0,adelphi,adelphi freedom perth
2,0,adelphi living jobs,adelphi freedom perth
3,0,the adelphi,adelphi freedom perth
4,0,freedom adelphi,adelphi freedom perth


In [12]:
# Merge it with the original dataset to get the matching query performances
final_df = pd.merge(df, clusters_queries, how="inner", on="Top queries")
final_df

,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name
0,adelphi living,289,834,34.65%,1.07,0,adelphi freedom perth
1,adelphi,15,298,5.03%,9.97,0,adelphi freedom perth
2,sil providers perth,12,90,13.33%,3.26,-1,living perth ndis
3,supported independent living perth,7,323,2.17%,10.77,1,independent living supported
4,adelphi living mandurah,7,30,23.33%,1.03,37,mandurah living access
...,...,...,...,...,...,...,...
713,disabled living manchester,0,1,0%,134.00,-1,living perth ndis
714,disability insurance perth,0,1,0%,141.00,24,disability perth insurance
715,specialist disability housing,0,1,0%,149.00,-1,living perth ndis
716,disability support providers,0,1,0%,199.00,13,disability support perth


### We now have a complete dataframe with our queries, performances and corresponding cluster names. 

### It means we can now perform data analysis per cluster, which can be useful especially if you have a lot of queries



## Export our dataframe to a Google Spreadsheet (previously created) so it can be read dynamically with Tableau and Google Data Studio

In [13]:
# Use of gspread to connect our Google Spreadsheet to our final dataframe
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

# Load credentials (not on the github / heroku folder, has to be replaced manually for safety purposes)
creds = Credentials.from_service_account_file("default.json", scopes=scope)
client = gspread.authorize(creds)

# Loading a Google Spreadsheet created before
google_sh = client.open("keyword_clustering")

# Selecting the sheet
sheet1 = google_sh.get_worksheet(0)

# Clearing output
sheet1.clear()

# Send our pandas dataframe to the google spreadsheet (you will be able to load it from Tableau and Google Data Studio)
df_google = set_with_dataframe(sheet1, final_df)

In [14]:
# Read our Google Spreadsheet to confirm the data has been correctly exported ('Unnamed' are the empty cells)
df_google = get_as_dataframe(sheet1)
df_google.head(20)

,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,adelphi living,289.0,834.0,0.3465,1.07,0.0,adelphi freedom perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,adelphi,15.0,298.0,0.0503,9.97,0.0,adelphi freedom perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sil providers perth,12.0,90.0,0.1333,3.26,-1.0,living perth ndis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,supported independent living perth,7.0,323.0,0.0217,10.77,1.0,independent living supported,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,adelphi living mandurah,7.0,30.0,0.2333,1.03,37.0,mandurah living access,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,disability accommodation perth,3.0,641.0,0.0047,31.52,18.0,accommodation disability services,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ndis providers near me,3.0,45.0,0.0667,1.00,2.0,near disability living,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,sda providers perth,3.0,23.0,0.1304,7.48,3.0,providers perth sda,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,disability housing perth,2.0,541.0,0.0037,31.37,4.0,housing disability perth,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ndis providers perth,2.0,258.0,0.0078,52.90,3.0,providers perth sda,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Export our final_df Dataframe to sqlite for further purposes (ex: load JSON URL data with Plotly or D3)

In [15]:
# SQL Alchemy
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [16]:
# Create connection
engine = create_engine('sqlite:///keyword_clustering.sqlite', echo=True)
conn = engine.connect()

2021-08-25 19:34:47,147 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-08-25 19:34:47,152 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:34:47,156 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-08-25 19:34:47,158 INFO sqlalchemy.engine.base.Engine ()


In [23]:
# Reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

2021-08-25 19:38:19,689 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-08-25 19:38:19,691 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:19,693 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-08-25 19:38:19,694 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:19,697 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-08-25 19:38:19,698 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:19,700 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("keyword_clustering")
2021-08-25 19:38:19,701 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:19,702 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("keyword_clustering")
2021-08-25 19:38:19,703 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:19,705 INFO sqlalchemy

In [24]:
# Export our dataframe to SQL and replace the data if the table is already there
final_df.to_sql("keyword_clustering", conn, if_exists="replace")

2021-08-25 19:38:21,266 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-08-25 19:38:21,267 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:21,269 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-08-25 19:38:21,271 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:21,272 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-08-25 19:38:21,273 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:21,275 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-08-25 19:38:21,276 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:21,279 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-08-25 19:38:21,280 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:21,282 INFO sqlalchemy.engine.base

In [25]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

2021-08-25 19:38:23,866 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-08-25 19:38:23,867 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:23,869 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_clustering")
2021-08-25 19:38:23,870 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:23,872 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'keyword_clustering' AND type = 'table'
2021-08-25 19:38:23,874 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:23,875 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("keyword_clustering")
2021-08-25 19:38:23,877 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:23,878 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("keyword_clustering")
2021-08-25 19:38:23,878 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:23,879 INFO sqlalchemy

[]

In [26]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [27]:
# Use `engine.execute` to select and display the first 10 rows from the station table
engine.execute('SELECT * FROM keyword_clustering LIMIT 5').fetchall()

2021-08-25 19:38:27,149 INFO sqlalchemy.engine.base.Engine SELECT * FROM keyword_clustering LIMIT 5
2021-08-25 19:38:27,150 INFO sqlalchemy.engine.base.Engine ()


[(0, 'adelphi living', 289, 834, '34.65%', 1.07, 0, 'adelphi freedom perth'),
 (1, 'adelphi', 15, 298, '5.03%', 9.97, 0, 'adelphi freedom perth'),
 (2, 'sil providers perth', 12, 90, '13.33%', 3.26, -1, 'living perth ndis'),
 (3, 'supported independent living perth', 7, 323, '2.17%', 10.77, 1, 'independent living supported'),
 (4, 'adelphi living mandurah', 7, 30, '23.33%', 1.03, 37, 'mandurah living access')]

In [28]:
# Query keyword_clustering in the the Database
data = pd.read_sql("SELECT * FROM keyword_clustering", conn)
data

2021-08-25 19:38:28,769 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM keyword_clustering")
2021-08-25 19:38:28,770 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:28,772 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM keyword_clustering")
2021-08-25 19:38:28,772 INFO sqlalchemy.engine.base.Engine ()
2021-08-25 19:38:28,774 INFO sqlalchemy.engine.base.Engine SELECT * FROM keyword_clustering
2021-08-25 19:38:28,775 INFO sqlalchemy.engine.base.Engine ()


,index,Top queries,Clicks,Impressions,CTR,Position,Cluster Number,Cluster Name
0,0,adelphi living,289,834,34.65%,1.07,0,adelphi freedom perth
1,1,adelphi,15,298,5.03%,9.97,0,adelphi freedom perth
2,2,sil providers perth,12,90,13.33%,3.26,-1,living perth ndis
3,3,supported independent living perth,7,323,2.17%,10.77,1,independent living supported
4,4,adelphi living mandurah,7,30,23.33%,1.03,37,mandurah living access
...,...,...,...,...,...,...,...,...
713,713,disabled living manchester,0,1,0%,134.00,-1,living perth ndis
714,714,disability insurance perth,0,1,0%,141.00,24,disability perth insurance
715,715,specialist disability housing,0,1,0%,149.00,-1,living perth ndis
716,716,disability support providers,0,1,0%,199.00,13,disability support perth
